# Simulations code


In [1]:
import numpy as np
from simulated_dipole import *
import matplotlib.pyplot as plt
import pandas as pd


## IN CASE OF SYBILL
mf = mass_fractions()
es = energy_spectrum()
smd = SMD_method()

## IN CASE OF EPOS
# mf = mass_fractions(model='EPOS')
# es = energy_spectrum()
# smd = SMD_method(model='EPOS')



# TO BE CHANGED WHETHER DNN OR UNIVERSALITY IS USED
keys_to_extract = ['dnn_xmax', 'energy', 'mass'] # energy in EeV
ddd = file_loader('AixNet_SYBILL') # remember to change also which key is used in `generate_xmax19()`



ddd = {kk: ddd[kk] for kk in keys_to_extract if kk in ddd} # to get a faster cycle
ddd = remove_nan_entries(ddd) # needed for KAne
ene_mask = (ddd['energy']>=8)
ddd = dict_cutter(ddd, ene_mask)
print(len(ddd['energy']))


print(ddd.keys())
print('Number of simulated events above 8 EeV:', len(ddd['energy']))


# All necessary items for random samples are here
n_samples = 1000
np.random.seed(90)
# these are the starting seeds for all the random generators, that are fixed and each different
seed_1 = np.arange(n_samples, 2*n_samples)
seed_2 = np.arange(2*n_samples,3*n_samples)
seed_3 = np.arange(3*n_samples,4*n_samples)
# seed_3 = np.arange(15,25) # just for testing


final_results = {'iter': [], 'emin': [], 'light': [], 'heavy': [], 'frac_l': [], 'frac_h': []}

237785
dict_keys(['dnn_xmax', 'energy', 'mass'])
Number of simulated events above 8 EeV: 237785


## Data production cycle

In [2]:
for k in range(n_samples):

    print('Iteration:', k+1, ' of ', n_samples)
    dd = ddd # here I make a copy, so that I can modify it (ddd stays the same for all cycles)
    print('Original length:', len(dd['energy']))

    # extract mass fractions
    mf.seed = seed_2[k] # fix seed
    mass_mask = mf.extract_all_fractions(np.log10(dd['energy'])+18, dd['mass']) # log(E/eV) is needed here
    dd = dict_cutter(dd, mass_mask)
    print('After mass fractions:', len(dd['energy']))

    # extract spectrum (bin-wise)
    es.seed = seed_3[k] # fix seed
    temp_dict = [{} for i in range(3)]
    for i in range(len(energies[:-1])):
        enne = energies[i]/1e18
        enneplus = energies[i+1]/1e18
        energy_mask = (dd['energy']>=enne)&(dd['energy']<enneplus)
        temp_dict[i] = dict_cutter(dd, energy_mask)
        spectrum_mask = es.spectrum_fraction(temp_dict[i]['energy'])
        temp_dict[i] = dict_cutter(temp_dict[i], spectrum_mask)

    dd = dict_paster(temp_dict)
    print('After energy spectrum:', len(dd['energy']))

    # define charge
    dd['charge'] = np.empty_like(dd['mass'])
    for mass in names:
        dd['charge'][np.where(dd['mass']==names[mass])]=charges[mass]

    generate_xmax19(dd, 'dnn_xmax', 'energy')
    # generate_lnA(dd, 'univ_xmax', 'univ_rmu', 'energy') # ONLY WORKS WITH UNIVERSALITY
    
    # estimate best quantiles
    thres_values = np.linspace(np.min(dd['xmax19']), np.max(dd['xmax19']), 27) # 25 values + 2 border values
    # thres_values = np.linspace(np.min(dd['lnA']), np.max(dd['lnA']), 27) # 25 values + 2 border values
    # print(thres_values)
    res = { 'emin': [], 
            'light': [], 
            'heavy': [], 
            'N_light': [],
            'N_heavy': [],
            'N_tot': [],
            'SMD': [],
            }
    
    for ee in range(len(energies)-1):
        e_min = energies[ee]/1e18
        e_max =energies[ee+1]/1e18


        # extract energy range
        e_mask = (dd['energy']<e_max)&(dd['energy']>=e_min)
        data = dict_cutter(dd, e_mask)

        for i in range(1, len(thres_values) - 2):
            thres_h = thres_values[i]

            for j in range(i + 1, len(thres_values) - 1):

                thres_l = thres_values[j]
                mask_h = (data['xmax19']<thres_h)
                mask_l = (data['xmax19']>thres_l)
                # mask_h = (data['lnA']<thres_h)
                # mask_l = (data['lnA']>thres_l)
                light_frac = dict_cutter(data, mask_l)
                heavy_frac = dict_cutter(data, mask_h)

                value = smd.quantify_SMD(light_frac['energy'], light_frac['charge'], heavy_frac['energy'], heavy_frac['charge'], d_max=1)
                A_l = np.sum(np.log(light_frac['mass']))
                A_h = np.sum(np.log(heavy_frac['mass']))

                res['emin'].append(e_min)
                res['light'].append(thres_l)
                res['heavy'].append(thres_h)
                res['SMD'].append(value)
                res['N_light'].append(len(light_frac['xmax19']))
                res['N_heavy'].append(len(heavy_frac['xmax19']))
                res['N_tot'].append(len(data['xmax19']))

    for key, value in res.items():
        res[key] = np.array(value)

    # format final results
    for l in range(len(energies[:-1])):

        emin_val = energies[l] / 1e18
        mask = res['emin'] == emin_val

        smd_vals = res['SMD'][mask]
        max_pos = np.argmax(smd_vals)

        valid_indices = np.where(mask)[0]

        idx = valid_indices[max_pos]
        n_tot = res['N_tot'][idx]
        n_light = res['N_light'][idx]
        n_heavy = res['N_heavy'][idx]

        if n_tot == 0:
            print(f"[WARNING] Total events is zero for energy bin {emin_val:.2f} EeV (iteration {k})")
            frac_l = np.nan
            frac_h = np.nan
        else:
            frac_l = n_light / n_tot
            frac_h = n_heavy / n_tot

        final_results['iter'].append(k)
        final_results['emin'].append(emin_val)
        final_results['light'].append(res['light'][idx])
        final_results['heavy'].append(res['heavy'][idx])
        final_results['frac_l'].append(frac_l)
        final_results['frac_h'].append(frac_h)




# Convert to pandas DataFrame
df_results = pd.DataFrame(final_results)
display(df_results)
df_results.to_csv("data_results/final_results_aixnet_sybill23d.csv", index=False)

Iteration: 1  of  1000
Original length: 237785
After mass fractions: 59436
After energy spectrum: 23295
Iteration: 2  of  1000
Original length: 237785
After mass fractions: 59224
After energy spectrum: 23151
Iteration: 3  of  1000
Original length: 237785
After mass fractions: 59414
After energy spectrum: 23256
Iteration: 4  of  1000
Original length: 237785
After mass fractions: 59280
After energy spectrum: 23497
Iteration: 5  of  1000
Original length: 237785
After mass fractions: 59453
After energy spectrum: 23297
Iteration: 6  of  1000
Original length: 237785
After mass fractions: 59461
After energy spectrum: 23445
Iteration: 7  of  1000
Original length: 237785
After mass fractions: 59207
After energy spectrum: 23449
Iteration: 8  of  1000
Original length: 237785
After mass fractions: 59401
After energy spectrum: 23370
Iteration: 9  of  1000
Original length: 237785
After mass fractions: 59435
After energy spectrum: 23227
Iteration: 10  of  1000
Original length: 237785
After mass fract

,iter,emin,light,heavy,frac_l,frac_h
0,0,8.0,795.022443,745.420815,0.284976,0.319356
1,0,16.0,778.488567,745.420815,0.306629,0.402484
2,0,32.0,778.488567,728.886939,0.188847,0.420814
3,1,8.0,798.682583,748.867760,0.254633,0.351397
4,1,16.0,782.077642,748.867760,0.287551,0.437578
...,...,...,...,...,...,...
2995,998,16.0,783.160903,750.065048,0.277935,0.436881
2996,998,32.0,766.612976,733.517121,0.255764,0.473411
2997,999,8.0,788.617879,747.444947,0.325234,0.332907
2998,999,16.0,788.617879,733.720636,0.235553,0.289817
